In [1]:
import requests
import os
import time
import bs4
import sqlite3
import pandas as pd

In [2]:
# url to the data
# The source of data is the Institute of Meteorology and Water Management - National Research Institute.
# Źródłem pochodzenia danych jest Instytut Meteorologii i Gospodarki Wodnej – Państwowy Instytut Badawczy.
url = r'https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/terminowe/synop/'

In [3]:
# function to find all links in a given page (except Parent Directory)
def get_links(url):
    page = bs4.BeautifulSoup(requests.get(url).content, 'html.parser')
    rows = page.find_all('tr')[3:-1]
    
    results = []
    
    for row in rows:
        href = row.find_all('td')[1].find('a').get('href')
        link = os.path.join(url, href)
        
        results.append(link)
        
    return results

In [4]:
# load file with metadata (column names in polish, data type, length) and extract names
# we need to store column names as files with measures do not have headers

data_format = []

with requests.get(url+'s_t_format.txt') as format_file:
    format_file.encoding = 'cp1250'
    data_format = [line.strip().split('  ')[0] for line in format_file.text.splitlines()[1:-8]]
    
for i, n in enumerate(data_format):
    print(i, n)

0 Kod stacji
1 Nazwa stacji
2 Rok
3 Miesiąc
4 Dzień
5 Godzina
6 Wysokość podstawy chmur CL CM szyfrowana [kod]
7 Status pomiaru HPOD
8 Wysokość podstawy niższej
9 Status pomiaru HPON
10 Wysokość podstawy wyższej [m]
11 Status pomiaru HPOW
12 Wysokość podstawy tekstowy [opis]
13 Pomiar przyrzadem 1 (niższa) [P]
14 Pomiar przyrzadem 2 (wyższa) [P]
15 Widzialność
16 Status pomiaru WID
17 Widzialność operatora [m]
18 Status pomiaru WIDO
19 Widzialność automat [m]
20 Status pomiaru WIDA
21 Zachmurzenie ogólne [oktanty]
22 Status pomiaru NOG
23 Kierunek wiatru
24 Status pomiaru KRWR
25 Prędkość wiatru
26 Status pomiaru FWR
27 Poryw wiatru
28 Status pomiaru PORW
29 Temperatura powietrza [°C]
30 Status pomiaru TEMP
31 Temperatura termometru zwilżonego [°C]
32 Status pomiaru TTZW
33 Wskaźnik wentylacji [W/N]
34 Wskaźnik lodu
35 Ciśnienie pary wodnej [hPa]
36 Status pomiaru CPW
37 Wilgotność względna [%]
38 Status pomiaru WLGW
39 Temperatura punktu rosy [°C]
40 Status pomiaru TPTR
41 Ciśnienie n

In [5]:
# connect ot SQLite database (or create if not exists)
# walk through all years and read all files, save them to database

print("Start...", time.strftime("%H:%M:%S", time.localtime()))
db = sqlite3.connect('./weather.db')

for year_url in get_links(url):
    print(year_url.split(url)[-1], end="\t")
    
    for file_url in get_links(year_url):
        print(".", end="")

        pd.read_csv(file_url, encoding='cp1250', names=data_format, low_memory=False)\
        .to_sql('weather', db, if_exists='append', index=False)
            
        time.sleep(1)
    print()
    
db.commit()
db.close()
print("Done!", time.strftime("%H:%M:%S", time.localtime()))

Start... 01:07:41
1960_1965/	.

/home/mint/.anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


....................
1966_1970/	............................................................
1971_1975/	............................................................
1976_1980/	.............................................................
1981_1985/	...............................................................
1986_1990/	...............................................................
1991_1995/	...............................................................
1996_2000/	.............................................................
2001/	.........................................................
2002/	.........................................................
2003/	.........................................................
2004/	.........................................................
2005/	.........................................................
2006/	.........................................................
2007/	.........................................................
2008/	..........